In [1]:
import os

from pathlib import Path

# Email Parser
from email import policy
from email.parser import BytesParser

from sklearn.model_selection import StratifiedKFold

import pandas as pd

In [32]:
NORMAL_DIR = Path('./') / "Phishing Dataset" / "normal"
PHISHING_DIR = Path('./') / "Phishing Dataset" / "phishing"

In [116]:
# Creating dataframe of required information

df = pd.DataFrame()

for file_name in os.listdir(NORMAL_DIR):

  with open(f"{NORMAL_DIR / file_name}", 'rb') as fp:
    f_path = fp.name  # Get file name
    mail = BytesParser(policy=policy.default).parse(fp)

  body = ''
  
  if mail.is_multipart():
    for part in mail.walk():
      ctype = part.get_content_type()
      cdispo = str(part.get('Content-Disposition'))

      if 'attachment' not in cdispo:
        body = part.get_payload()
  else:
    body = mail.get_payload()
    
  subject = mail.get_all('subject')
  from_ = mail.get_all('from')
  content_type = mail.get_all('content-type')

  temp_df = pd.DataFrame({
    'label': 0,
    'from': from_,
    'content_type': content_type,
    'f_path': f_path,
    'subject': subject,
    'body': body
  }, index=[0])

  df = pd.concat([df, temp_df], ignore_index=True)

df.to_csv('normal_mails.csv', index_label=False)

In [136]:
# .DS_Store file if there is any
df = df.drop(380)

In [139]:
df1 = pd.DataFrame()

for file_name in os.listdir(PHISHING_DIR):

  with open(f"{PHISHING_DIR / file_name}", 'rb') as fp:
    f_path = fp.name  
    mail = BytesParser().parse(fp)

  body = ''
  
  if mail.is_multipart():
    for part in mail.walk():
      ctype = part.get_content_type()
      cdispo = str(part.get('Content-Disposition'))

      if 'attachment' not in cdispo:
        body = part.get_payload()  # decode
  else:
    body = mail.get_payload()

  subject = mail['subject']
  from_ = mail['from']
  content_type = mail['content-type']

  temp_df = pd.DataFrame({
    'label': 1,
    'from': from_,
    'content_type': content_type,
    'f_path': f_path,
    'subject': subject,
    'body': body
  }, index=[0])

  df1 = pd.concat([df1, temp_df], ignore_index=True)

df1.to_csv('phishing_mails.csv', index_label=False)

In [140]:
df = pd.concat([df, df1], ignore_index=True)

In [141]:
df = df.sample(frac=1, random_state=41).reset_index(drop=True)

In [145]:
df.to_csv('dataset.csv', index_label=False)

In [142]:
# skf = StratifiedKFold(n_splits=3)
# for fold, (_, val) in enumerate(skf.split(X=df, y=df.label)):
#   df.loc[val, "kfold"] = fold

In [143]:
# train_df = df[(df.kfold==0.0) | (df.kfold==2.0)]
# test_df = df[(df.kfold==1.0)]

In [144]:
# train_df.reset_index(drop=True).to_csv('train_df.csv', index_label=False)
# test_df.reset_index(drop=True).to_csv('test_df.csv', index_label=False)